# Computing chemical properties

So far, we have gained understanding of how to work with [ChemicalSystem](https://reaktoro.org/api/classReaktoro_1_1ChemicalSystem.html) and [ChemicalState](https://reaktoro.org/api/classReaktoro_1_1ChemicalState.html). Below, we continue learning to evaluate *thermodynamic and chemical properties of the system, its phases, and constituting species* using class [ChemicalProps](https://reaktoro.org/api/classReaktoro_1_1ChemicalProps.html).

Let's start with a simple chemical system containing a single aqueous phase and chemical state representing a 1 molal NaCl solution with 0.7 molal dissolved CO<sub>2</sub>:

In [1]:
from reaktoro import *

db = PhreeqcDatabase("phreeqc.dat")

solution = AqueousPhase(speciate("H O Na Cl C"))

system = ChemicalSystem(db, solution)

state = ChemicalState(system)
state.temperature(60.0, "celsius")
state.pressure(15.0, "atm")
state.set("H2O", 1.0, "kg")
state.set("Na+", 1.0, "mol")
state.set("Cl-", 1.0, "mol")
state.set("CO2", 0.7, "mol")

equilibrate(state)

print(state)

+-----------------+-------------+------+
| Property        |       Value | Unit |
+-----------------+-------------+------+
| Temperature     |    333.1500 |    K |
| Pressure        |     15.1988 |  bar |
| Charge:         | -1.0371e-16 |  mol |
| Element Amount: |             |      |
| :: H            |  1.1101e+02 |  mol |
| :: C            |  7.0000e-01 |  mol |
| :: O            |  5.6906e+01 |  mol |
| :: Na           |  1.0000e+00 |  mol |
| :: Cl           |  1.0000e+00 |  mol |
| Species Amount: |             |      |
| :: CO3-2        |  4.9431e-11 |  mol |
| :: H+           |  7.1562e-04 |  mol |
| :: H2O          |  5.5506e+01 |  mol |
| :: CO2          |  6.9928e-01 |  mol |
| :: (CO2)2       |  1.0000e-16 |  mol |
| :: HCO3-        |  4.8462e-04 |  mol |
| :: CH4          |  1.0000e-16 |  mol |
| :: Cl-          |  1.0000e+00 |  mol |
| :: H2           |  1.0000e-16 |  mol |
| :: Na+          |  9.9977e-01 |  mol |
| :: NaCO3-       |  4.4527e-09 |  mol |
| :: NaHCO3     

> **Note**: Above, we use a convenience method [equilibrate](https://reaktoro.org/api/namespaceReaktoro.html#a286e2972325304934d631234abef9e87) to quickly bring the [ChemicalState](https://reaktoro.org/api/classReaktoro_1_1ChemicalState.html) object `state` into a state of chemical equilibrium. It is implemented for the case, when the Gibbs energy minimization has to be performed once. However, for the sequence of chemical equilibrium calculations, we recommend using class [EquilibriumSolver](https://reaktoro.org/api/classReaktoro_1_1EquilibriumSolver.html), which
> * improves the performance (avoiding initialization of new equilibrium solver at each call of `equilibrate`)
> * add flexibility to enforce different equilibrium constraints at equilibrium (e.g., fixed pH, pE, Eh, gas fugacity, enthalpy, internal energy, volume, etc.) in combination classes [EquilibriumSpecs](https://reaktoro.org/api/classReaktoro_1_1EquilibriumSpecs.html) and [EquilibriumConditions](https://reaktoro.org/api/classReaktoro_1_1EquilibriumConditions.html).

Generally speaking, computing chemical properties of [ChemicalSystem](https://reaktoro.org/api/classReaktoro_1_1ChemicalSystem.html) objects is a task for class [ChemicalProps](https://reaktoro.org/api/classReaktoro_1_1ChemicalProps.html).

In [2]:
props = ChemicalProps(state)
print(props)

+----------------------------------------+--------------+-----------+
| Property                               |        Value |      Unit |
+----------------------------------------+--------------+-----------+
| Temperature                            |     333.1500 |         K |
| Pressure                               |      15.1988 |       bar |
| Volume                                 |   1.0595e-03 |        m3 |
| Gibbs Energy                           |  -73283.1995 |         J |
| Enthalpy                               |   -1608.0387 |         J |
| Entropy                                |     215.1438 |       J/K |
| Internal Energy                        |   -3218.3323 |         J |
| Helmholtz Energy                       |  -74893.4931 |         J |
| Charge                                 |  -1.0371e-16 |       mol |
| Element Amount:                        |              |           |
| :: H                                   |   1.1101e+02 |       mol |
| :: C              

**Note**: `ActivityCoefficients` above are all set to 1.0, since all phases in chemical system are assigned with **ideal activity models** by default.

For comparison, we reconstruct the [ChemicalSystem](https://reaktoro.org/api/classReaktoro_1_1ChemicalSystem.html) with Pitzer's activity model for aqueous phase and activity model based on Peng-Robinson's equation of state for the gaseous phase as well as reevaluate chemical state and its properties:

In [3]:
solution = AqueousPhase(speciate("H O Na Cl C Ca Si"))
solution.setActivityModel(ActivityModelPitzerHMW())

gases = GaseousPhase()
gases.setActivityModel(ActivityModelPengRobinson())

minerals = MineralPhases()

system = ChemicalSystem(db, solution, gases, minerals)

state = ChemicalState(system)
state.temperature(60.0, "celsius")
state.pressure(15.0, "atm")
state.set("H2O"    , 1.0, "kg")
state.set("Na+"    , 1.0, "mol")
state.set("Cl-"    , 1.0, "mol")
state.set("CO2"    , 0.7, "mol")
state.set("Calcite", 1.0, "g")
state.set("Quartz" , 1.0, "g")

equilibrate(state)

# Chemical properties can be also fetched from the chemical state as they get updated at the end of the optimization algorithm
props = state.props()

print(props)

+----------------------------------------+--------------+-----------+
| Property                               |        Value |      Unit |
+----------------------------------------+--------------+-----------+
| Temperature                            |     333.1500 |         K |
| Pressure                               |      15.1988 |       bar |
| Volume                                 |   1.9586e-03 |        m3 |
| Gibbs Energy                           |  -79556.4911 |         J |
| Enthalpy                               |    5288.4193 |         J |
| Entropy                                |     254.6748 |       J/K |
| Internal Energy                        |    2311.6270 |         J |
| Helmholtz Energy                       |  -82533.2834 |         J |
| Charge                                 |  -2.1583e-16 |       mol |
| Element Amount:                        |              |           |
| :: H                                   |   1.1101e+02 |       mol |
| :: C              

The results displayed above should now be a little more interesting.

Now consider a scenario where chemical state has changed, for example 1 mol of NaCl has been added to it and the temperature has increased by 10 °C. The chemical properties of the system must then be updated as shown below:

In [4]:
state.temperature(70.0, "celsius")
state.add("Na+", 1.0, "mol")
state.add("Cl-", 1.0, "mol")

props.update(state)

print(props)

+----------------------------------------+--------------+-----------+
| Property                               |        Value |      Unit |
+----------------------------------------+--------------+-----------+
| Temperature                            |     343.1500 |         K |
| Pressure                               |      15.1988 |       bar |
| Volume                                 |   1.9569e-03 |        m3 |
| Gibbs Energy                           |  -82231.0103 |         J |
| Enthalpy                               |    8403.5580 |         J |
| Entropy                                |     264.1252 |       J/K |
| Internal Energy                        |    5429.3260 |         J |
| Helmholtz Energy                       |  -85205.2423 |         J |
| Charge                                 |  -2.1583e-16 |       mol |
| Element Amount:                        |              |           |
| :: H                                   |   1.1101e+02 |       mol |
| :: C              

The printed table above shows a long list of properties. Below, we use some methods from class [ChemicalProps](https://reaktoro.org/api/classReaktoro_1_1ChemicalProps.html) to retrieve specific properties for the system, its phases, and its species:

In [5]:
print(f"SYSTEM'S TEMPERATURE     : {props.temperature()                        } K    ")
print(f"SYSTEM'S PRESSURE        : {props.pressure()                           } Pa   ")
print(f"SYSTEM'S VOLUME          : {props.volume()                             } m3   ")
print(f"AQUEOUS PHASE'S DENSITY  : {props.phaseProps('AqueousPhase').density() } kg/m3")
print(f"AQUEOUS PHASE'S MASS     : {props.phaseProps('AqueousPhase').mass()    } kg   ")
print(f"AQUEOUS PHASE'S VOLUME   : {props.phaseProps('AqueousPhase').volume()  } m3   ")
print(f"AQUEOUS PHASE'S ENTHALPY : {props.phaseProps('AqueousPhase').enthalpy()} J/mol")
print(f"GASEOUS PHASE'S DENSITY  : {props.phaseProps('GaseousPhase').density() } kg/m3")
print(f"GASEOUS PHASE'S MASS     : {props.phaseProps('GaseousPhase').mass()    } kg   ")
print(f"GASEOUS PHASE'S VOLUME   : {props.phaseProps('GaseousPhase').volume()  } m3   ")
print(f"GASEOUS PHASE'S ENTHALPY : {props.phaseProps('GaseousPhase').enthalpy()} J/mol")
print(f"ACTIVITY H+              : {props.speciesActivity('H+')                }      ")
print(f"ACTIVITY H+ (LOG10)      : {props.speciesActivityLg('H+')              }      ")
print(f"ACTIVITY COEFF. CO2      : {props.speciesActivityCoefficient('CO2')    }      ")
print(f"ACTIVITY COEFF. HCO3-    : {props.speciesActivityCoefficient('HCO3-')  }      ")
print(f"FUGACITY COEFF. CO2(g)   : {props.speciesActivityCoefficient('CO2(g)') }      ")
print(f"FUGACITY COEFF. H2O(g)   : {props.speciesActivityCoefficient('H2O(g)') }      ")
print(f"FUGACITY COEFF. O2(g)    : {props.speciesActivityCoefficient('O2(g)')  }      ")

SYSTEM'S TEMPERATURE     : 343.15 K    
SYSTEM'S PRESSURE        : 1.51988e+06 Pa   
SYSTEM'S VOLUME          : 0.00195689 m3   
AQUEOUS PHASE'S DENSITY  : 1058.56 kg/m3
AQUEOUS PHASE'S MASS     : 1.12676 kg   
AQUEOUS PHASE'S VOLUME   : 0.00106443 m3   
AQUEOUS PHASE'S ENTHALPY : 642.393 J/mol
GASEOUS PHASE'S DENSITY  : 24.6099 kg/m3
GASEOUS PHASE'S MASS     : 0.0219544 kg   
GASEOUS PHASE'S VOLUME   : 0.000892096 m3   
GASEOUS PHASE'S ENTHALPY : 8170.97 J/mol
ACTIVITY H+              : 1.51131e-05      
ACTIVITY H+ (LOG10)      : -4.82065      
ACTIVITY COEFF. CO2      : 1.37659      
ACTIVITY COEFF. HCO3-    : 0.557782      
FUGACITY COEFF. CO2(g)   : 0.947625      
FUGACITY COEFF. H2O(g)   : 0.866819      
FUGACITY COEFF. O2(g)    : 1.0112      


To summarise, [ChemicalProps](https://reaktoro.org/api/classReaktoro_1_1ChemicalProps.html) class stores *general thermodynamic and chemical properties for systems, their phases and species* without any assumptions about their content.
